In [54]:
# models = [
#     "gpt-4o-12q",
#     "granite-3-3-8b-6q",
#     "llama-3-2-3b-6q",
#     "mistral-nemo-6q",
#     "mistral-nemo-12q",
#     "mistral-7b-0-1-6q",
#     "mistral-7b-0-1-12q",
#     "mistral-7b-0-3-6q"
#     ]


models = [
    "granite-3-3-8b-6q",
    "llama-3-2-3b-6q",
    "mistral-nemo-6q",
    "mistral-7b-0-1-6q",
    "mistral-nemo-post-6q"
    ]

In [55]:
import pandas as pd
precision_recall_excel_sheet_template = {
    'name': '',
    'precision_recall_df': pd.DataFrame()
}

In [56]:
import pandas as pd
excel_sheet_template = {
    'name': '',
    'acc_data': pd.DataFrame(),
    'val_data': pd.DataFrame()
}

In [57]:
columns_12 = [
        'overall_sentiment', 'nature_dimension', 'nature_sentiment', 'functional_dimension', 'functional_sentiment',
        'normative_dimension', 'normative_sentiment', 'cultural_dimension', 'cultural_sentiment',
        'disinformation_type', 'disinformation_technique'
    ]

columns_6 = [
        'overall_sentiment',  'nature_sentiment',  'functional_sentiment', 'normative_sentiment', 'cultural_sentiment',
        'disinformation_type'
    ]

columns = []

In [58]:
santements = ['very negative', 'negative', 'neutral', 'positive', 'very positive', 'No Sentiment']

natural_dimensions = ['Nature Dimension not addressed', 'Landscape/scenery', 'Geography', 'Weather/climate', 'Preserved nature', 'Nature activities', 'Other aspect of Nature dimension']
functional_dimensions = ['Functional Dimension not addressed', 'Education system', 'Science/innovation', 'Products', 'Economy', 'Infrastructure', 'Politics', 'Living/working conditions', 'Security', 'Other aspect of Functional dimension']
normative_dimensions = ['Normative Dimension not addressed', 'Environmental protection', 'Freedom/human rights', 'Civil rights', 'International engagement', 'Ethical issues/scandals', 'Conflict avoidance', 'Tolerance/openness', 'Other aspect of Normative dimension']
cultural_dimensions = ['Cultural Dimension not addressed', 'Sports', 'Food', 'Cultural offer', 'Personalities', 'Traditions', 'History', 'Cultural diversity', 'Other aspect of Culture dimension']
disinformation_type = ['No disinformation type', 'False connection', 'False context', 'Misleading content', 'Fabricated content', 'Manipulated content', 'Other disinformation type']
disinformation_technique = ['No disinformation technique', 'Ad hominem attack', 'Emotional language', 'False dichotomies', 'Incoherence', 'Scapegoating', 'Other disinformation technique']

valid_options = santements + natural_dimensions + functional_dimensions + normative_dimensions + cultural_dimensions + disinformation_type + disinformation_technique

valid_options = [x.lower().strip().rstrip('.') for x in valid_options]

In [59]:
len(valid_options)

55

In [60]:
def find_labels(col_name):
    match col_name:
        case "overall_sentiment":
            return santements
        case "nature_dimension":
            return natural_dimensions
        case "nature_sentiment":
            return santements
        case "functional_dimension":
            return functional_dimensions
        case "functional_sentiment":
            return santements
        case "normative_dimension":
            return normative_dimensions
        case "normative_sentiment":
            return santements
        case "cultural_dimension":
            return cultural_dimensions
        case "cultural_sentiment":
            return santements
        case "disinformation_type":
            return disinformation_type
        case "disinformation_technique":
            return disinformation_technique
        case _:
            return []

In [61]:
def string_normalization(str_value):
    if isinstance(str_value, str):
        return str_value.lower().strip().rstrip('.')
    else:
        return ""


In [62]:
def is_valid(input_str):
    if string_normalization(input_str) in valid_options:
        return True
    else:
        return False

In [63]:
from pathlib import Path
# question_model_path = 'question_model_1'
question_model_path = 'question_model_2'
# question_model_path = 'question_model_2_with_parameters'

def get_models_files(model_path):
    try:
        path = Path(f'data/{question_model_path}/{model_path}')
        file_list = [f.name for f in path.iterdir() if f.is_file()]
        prefix = f"data/{question_model_path}/{model_path}/"
        prefixed_file_list = [prefix + s for s in file_list]
        return prefixed_file_list
    except Exception as e:
        print("An unexpected error occurred in get_models_files:", e)
        return []

In [64]:
# import pandas as pd
# def print_line():
#     print("--------------------------------------------")
    
input_file_path = f'data/final_file_mistral_nemo__post_1_133_2025-06-20_14-53-44.csv'
llm_values = pd.read_csv(input_file_path)
referance_file_path = 'data/valid_test_final_file.csv'
referance_values = pd.read_csv(referance_file_path)

if len(llm_values.columns) == 22:
    columns = columns_12
else:
    columns = columns_6

In [65]:
# precision and recall

from copy import deepcopy


labels = []

precision_recall_excel_sheets = []

for model in models:
    file_list = get_models_files(model_path=model)
    precision_recall_df = pd.DataFrame(columns=[
        'question',
        'label',
        'precision',
        'recall',
        'TP',
        'FP',
        'FN',
        'exp_id'
        ])
    for file_index, file_path in enumerate(file_list):
        llm_values = pd.read_csv(file_path)
        if len(llm_values.columns) == 22:
            columns = columns_12
        else:
            columns = columns_6

        for col in columns:
            y_pred = llm_values[col]
            y_ref = referance_values[col]
            labels = find_labels(col)

            for label in labels:
                tp = 0  # True Positives
                fp = 0  # False Positives
                fn = 0  # False Negatives
                for i in range(len(y_pred)):
                    predicted = string_normalization(y_pred[i])
                    actual = string_normalization(y_ref[i])
                    val_label = string_normalization(label)

                    if predicted == val_label and actual == val_label:
                        tp += 1
                    elif predicted == val_label and actual != val_label:
                        fp += 1
                    elif predicted != val_label and actual == val_label:
                        fn += 1

                # Calculate precision and recall
                if tp + fp > 0:
                    precision = round(tp*100 / (tp + fp),2)
                else:
                    precision = 0.00

                if tp + fn > 0:
                    recall = round(tp*100 / (tp + fn),2)
                else:
                    recall = 0.00

                precision_recall_df.loc[len(precision_recall_df)] = [col, label, precision, recall, tp, fp, fn, file_index]
                
        
    excel_sheet = deepcopy(precision_recall_excel_sheet_template)
    excel_sheet['name'] = model
    excel_sheet['precision_recall_df'] = precision_recall_df
    precision_recall_excel_sheets.append(excel_sheet)

with pd.ExcelWriter('precision_recall_all.xlsx', engine='xlsxwriter') as writer:
    for sheet in precision_recall_excel_sheets:
        sheet_name = sheet['name']
        sheet['precision_recall_df'].to_excel(writer, sheet_name=f"{sheet_name}", index=False)
        



In [66]:
# all accuracies

from copy import deepcopy

excel_sheets = []

for model in models:
    file_list = get_models_files(model_path=model)
    val_df = pd.DataFrame()
    acc_df = pd.DataFrame()
    for file_index, file_path in enumerate(file_list):
        llm_values = pd.read_csv(file_path)
        if len(llm_values.columns) == 22:
            columns = columns_12
        else:
            columns = columns_6

        if "Columns" not in val_df.columns:
            val_df["Columns"] = columns

        if "Columns" not in acc_df.columns:
            acc_df["Columns"] = columns

        val_df[f'Exp_{file_index+1}'] = 0.0
        
        for col in columns:

            valid_values = 0
            accurate_values = 0
            
            for index, row in llm_values.iterrows():
                try:
                    value = llm_values.at[index, col]
                    if isinstance(value, str):
                        if is_valid(value):
                            valid_values += 1
                except Exception as e:
                    print("An unexpected error occurred col:", e)
            val_df.loc[val_df['Columns'] == col, f'Exp_{file_index+1}'] = round(valid_values*100/len(llm_values),2)


            accurate_values = 0
            for index, row in referance_values.iterrows():
                try:
                    referance_value = referance_values.at[index, col]
                    llm_value = llm_values.at[index, col]
                    if isinstance(referance_value, str) and isinstance(llm_value, str):
                        referance_value = referance_value.lower().strip().rstrip('.')
                        llm_value = llm_value.lower().strip().rstrip('.')
                        if referance_value == llm_value:
                            accurate_values += 1
                except Exception as e:
                    pass
            acc_df.loc[acc_df['Columns'] == col, f'Exp_{file_index+1}'] = round(accurate_values*100/len(llm_values),2)

    acc_df['Mean'] = acc_df[['Exp_1', 'Exp_2', 'Exp_3', 'Exp_4', 'Exp_5']].mean(axis=1).round(2)
    acc_df['StD'] = acc_df[['Exp_1', 'Exp_2', 'Exp_3', 'Exp_4', 'Exp_5']].std(axis=1).round(2)

    val_df['Mean'] = val_df[['Exp_1', 'Exp_2', 'Exp_3', 'Exp_4', 'Exp_5']].mean(axis=1).round(2)
    val_df['StD'] = val_df[['Exp_1', 'Exp_2', 'Exp_3', 'Exp_4', 'Exp_5']].std(axis=1).round(2)

    excel_sheet = deepcopy(excel_sheet_template)
    excel_sheet['name'] = model
    excel_sheet['acc_data'] = acc_df
    excel_sheet['val_data'] = val_df
    excel_sheets.append(excel_sheet)

pass
with pd.ExcelWriter('all_accuracy.xlsx', engine='xlsxwriter') as writer:
    for sheet in excel_sheets:
        sheet_name = sheet['name']
        sheet['val_data'].to_excel(writer, sheet_name=f"val_{sheet_name}", index=False)
        sheet['acc_data'].to_excel(writer, sheet_name=f"acc_{sheet_name}", index=False)

In [40]:
print(f'Data length: {len(llm_values)}')
all_vals = 0
for col in columns:
    valid_values = 0
    for index, row in llm_values.iterrows():
        try:
            value = llm_values.at[index, col]
            if isinstance(value, str):
                if is_valid(value):
                    valid_values += 1
        except Exception as e:
            pass
    val = round(valid_values*100/len(llm_values),2)
    all_vals += val
    print(f"Valid {col}: {val} %.")

print(f"Avg validity: {round(all_vals/len(columns),2)} %.")


Data length: 133
Valid overall_sentiment: 94.74 %.
Valid nature_sentiment: 84.21 %.
Valid functional_sentiment: 84.21 %.
Valid normative_sentiment: 84.21 %.
Valid cultural_sentiment: 84.21 %.
Valid disinformation_type: 84.21 %.
Avg validity: 85.96 %.


In [41]:
all_accuracies = 0
for col in columns:
    accurate_values = 0
    for index, row in referance_values.iterrows():
        try:
            referance_value = referance_values.at[index, col]
            llm_value = llm_values.at[index, col]
            if isinstance(referance_value, str) and isinstance(llm_value, str):
                referance_value = referance_value.lower().strip().rstrip('.')
                llm_value = llm_value.lower().strip().rstrip('.')
                if referance_value == llm_value:
                    accurate_values += 1
        except Exception as e:
            pass
    accuracy = round(accurate_values*100/len(referance_values),2)        
    all_accuracies += accuracy
    
    print(f"{col} accuracy: {accuracy} %.")

print(f"Avg accuracy: {round(all_accuracies/len(columns),2) } %.")


overall_sentiment accuracy: 42.86 %.
nature_sentiment accuracy: 64.66 %.
functional_sentiment accuracy: 41.35 %.
normative_sentiment accuracy: 39.1 %.
cultural_sentiment accuracy: 48.12 %.
disinformation_type accuracy: 78.2 %.
Avg accuracy: 52.38 %.
